In [1]:
# Feature engineering and model selection
from sklearn.pipeline import Pipeline

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')

# Analytical helper functions to declutter the notebook
from analysis.ml.custom_transformers import *
from analysis.ml.data import get_ml_features, get_ml_dataset

ModuleNotFoundError: No module named 'analysis'

In [2]:
# Construct base datasets for model training
get_ml_dataset()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(SPOT) Processing asset 1 of 601 (1000SATS/FDUSD on BINANCE)...
(SPOT) Processing asset 2 of 601 (1000SATS/USDC on BINANCE)...
(SPOT) Processing asset 3 of 601 (1000SATS/USDT on BINANCE)...
(SPOT) Processing asset 4 of 601 (1INCH/USDT on BINANCE)...
(SPOT) Processing asset 5 of 601 (1MBABYDOGE/FDUSD on BINANCE)...
(SPOT) Processing asset 6 of 601 (1MBABYDOGE/USDT on BINANCE)...
(SPOT) Processing asset 7 of 601 (AAVE/FDUSD on BINANCE)...
(SPOT) Processing asset 8 of 601 (AAVE/USDC on BINANCE)...
(SPOT) Processing asset 9 of 601 (AAVE/USDT on BINANCE)...
(SPOT) Processing asset 10 of 601 (ACA/USDT on BINANCE)...
(SPOT) Processing asset 11 of 601 (ACE/USDT on BINANCE)...
(SPOT) Processing asset 12 of 601 (ACH/USDT on BINANCE)...
(SPOT) Processing asset 13 of 601 (ACM/USDT on BINANCE)...
(SPOT) Processing asset 14 of 601 (ADA/FDUSD on BINANCE)...
(SPOT) Processing asset 15 of 601 (ADA/USDC on BINANCE)...
(SPOT) Processing asset 16 of 601 (ADA/USDT on BINANCE)...
(SPOT) Processing asset 17 

NameError: name 'dataset' is not defined

In [ ]:
# Pipeline for feature engineering and modeling
feature_engineering_pipeline = Pipeline([

    ('time_features', TimeFeatures()),

    ('returns_features', ReturnsFeatures(
        window_sizes = [1, 7, 30, 180],
        lookback_windows = [7, 30, 180],
    )),

    ('risk_features', RiskFeatures(
        windows = [1],
        lookback_windows = [30, 180]
    )),
    #
    ('trade_features', TradeFeatures(
        windows = [1, 7, 30, 90, 180, 365],
        lookback_windows = [7, 30, 180]
    )),

    ('fillna', FillNaTransformer()),

    ('rolling_z_score', RollingZScoreScaler(window_sizes = [30, 90, 180, 365])),

])

In [6]:
print(trading_bot.__file__)


NameError: name 'trading_bot' is not defined

In [ ]:
# Pipeline for feature engineering and modeling
feature_engineering_pipeline = Pipeline([

    ('time_features', TimeFeatures()),

    ('returns_features', ReturnsFeatures(
        window_sizes = [1, 7, 30, 180],
        lookback_windows = [7, 30, 180],
    )),

    ('risk_features', RiskFeatures(
        windows = [1],
        lookback_windows = [30, 180]
    )),
    #
    ('trade_features', TradeFeatures(
        windows = [1, 7, 30, 90, 180, 365],
        lookback_windows = [7, 30, 180]
    )),

    ('fillna', FillNaTransformer()),

    ('rolling_z_score', RollingZScoreScaler(window_sizes = [30, 180])),

])

In [ ]:
get_ml_features(feature_engineering_pipeline = feature_engineering_pipeline)

Processing symbol_id: 1000SATS_FDUSD_BINANCE (1/467)
Binder Error: table ml_features has 2437 columns but 5005 values were supplied
Processing symbol_id: 1000SATS_USDC_BINANCE (2/467)
Column set difference: set()

Processing symbol_id: 1000SATS_USDT_BINANCE (3/467)
Column set difference: set()

Processing symbol_id: 1INCH_USDT_BINANCE (4/467)
Column set difference: set()

Processing symbol_id: 1MBABYDOGE_FDUSD_BINANCE (5/467)
Column set difference: set()

Processing symbol_id: 1MBABYDOGE_USDT_BINANCE (6/467)
Column set difference: set()

Processing symbol_id: AAVE_FDUSD_BINANCE (7/467)
Column set difference: set()

Processing symbol_id: AAVE_USDC_BINANCE (8/467)
Column set difference: set()

Processing symbol_id: AAVE_USDT_BINANCE (9/467)
Column set difference: set()

Processing symbol_id: ACA_USDT_BINANCE (10/467)
Column set difference: set()

